# LLama.CPP
LLama.CPP is a tool that enables us to run LLM inference on a local machine. This is true for both desktop and mobile (which is our main focus). \
In this notebook I present results of my research on this tool; how we can use, where we can use it, and what are its limitations.

# 1. Build Llama.cpp on Desktop
For obvious reasons (such as physical keyboard, mouse etc), experimenting with Llama.cpp is much easier on desktop. \
So while we target mobile platforms, for learning and research purposes I worked with the desktop version, \
and so here are the steps to run Llama.cpp on desktop (here I present steps for Linux, but Windows version is also available in the repository docs):
1. Clone the Llama.cpp repository, which can be found here: https://github.com/ggerganov/llama.cpp
2. Install CMake (if you don't have it already) by running `sudo apt install cmake`
3. Install Clang compiler by running `sudo apt install clang`
4. (Optional) On mobile i install libllvm instead (it should contain clang and cmake). I didn't try this, but you can.
5. Enter the llama.cpp directory and create a build directory by running `cmake -B build`
6. Build the project by running `cmake --build build --config Release`


# 2. Run LLM on Desktop
There are actually three ways to run LLMs (not true for BERTs, I will describe this later). For each of them,
you will need a model saved in a .gguf format (which is a serialized model format used by Llama.cpp). \
TheBloke user on Hugging Face has shared a lot of models converted to this format; you can find him here: https://huggingface.co/TheBloke.
For the purpose of examples here, I will use TheBloke/Llama-2-7B-GGUF (https://huggingface.co/TheBloke/Llama-2-7B-GGUF).

## 1. Run LLM directly from the command line
First of all, you need to download the model. When you hed to the model page and its files, it will contain \
a lot of different versions with different quantizations (described on the model's page). \
I will use the Q5 version. Now, you can download and store it anywhere, but for performance reasons \
(at least on android) it is suggested to store it under ~/ path.\
Now, you can run the model by running the following command:
```bash
./build/bin/llama-simple -m ~/llama-2-7b.Q5_K_S.gguf -c 4096 -p "What is the capital of France?"
```
Now, it is a big model, and running it on CPU may take a while. But, if everything goes smoothly, you should \
get something similar to this: \
![llama-7B](./ResearchImages/llama7bdesktop.png)

## 2. Run LLM as a server
Llama.cpp has a basic server for running with LLMs (and it actually has a nice API, so we can write scripts to communicate with it). \
To run the server, you need to run the following command:
```bash
./build/bin/llama-server -m ~/llama-2-7b.Q5_K_S.gguf -c 4096
```
After that you can go under http://127.0.0.1:8080/ and you should see the server's page: \
![llama-server](./ResearchImages/LLamacppserver.png)
In this particular case, results are very bad. But it is dependent on the model, I played with smaller ones \
and while responses were not perfect, they were making sense and were actually related to the topic.


# 2. Build Llama.cpp on Mobile

This is actually a really fun part. To run Llama.cpp on mobile, you need a terminal, and Termux is all you need.  
It is a terminal emulator for Android, and you can find it here: [https://termux.dev/en/](https://termux.dev/en/).  
You can download it directly from the Play Store, but for the newest version, I suggest getting it from F-Droid (I did it both ways).  
To get it from F-Droid, follow the steps described on the project's page.  

After you install Termux, you have to basically repeat the steps from the desktop version.  
First of all, you need to install:
- **Git** by running `pkg install git`
- **libllvm** by running `pkg install libllvm` (this should contain both clang and cmake)

Now, steps are almost identical to the desktop version:
1. Clone the Llama.cpp repository by running `git clone https://github.com/ggerganov/llama.cpp'
2. Enter the llama.cpp directory by running `cd llama.cpp`
3. Create a build directory by running `cmake -B build`
4. Build the project by running `cmake --build build --config Release`

    

# 3. Run LLM on Mobile
There are two ways to run LLMs on mobile, and they are actually the same as on desktop:

## 1. Run LLM directly from the command line
This time, we will use a smaller model (but you can also play with the 7b llama, it works). \
To get the model, we will uses curl for the ease of use: \
```bash
curl -L -o tinyllama-1.1b-chat-v0.3.Q5_K_S.gguf https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v0.3-GGUF/resolve/main/tinyllama-1.1b-chat-v0.3.Q5_K_S.gguf
```
This should download the file and save it under your current location so before doing this, I suggest you enter ~/ dir ('cd ..' from llama.cpp folder). \

Now, to run the model, you can run the following command:
```bash
./build/bin/llama-simple -m ~/tinyllama-1.1b-chat-v0.3.Q5_K_S.gguf -c 4096 -p "What is the capital of France?"
```

And you should get a response similar to this: \
![tinyllama-1.1B](./ResearchImages/tiny_termux.png)

One thing you can observe, is that while the response is far from perfect, it is related to the question.
And it was really fast (almost instant).

## 2. Run LLM as a server
To run the server, you need to run the following command:
```bash
./build/bin/llama-server -m ~/tinyllama-1.1b-chat-v0.3.Q5_K_S.gguf -c 4096
```
After that you can go under http://127.0.0.1:8080/ and you should see the server's page: \
![llama-server](./ResearchImages/termux_server.png) \
As you can see, the app is not fitted for mobile, but it works. Now, main problem with this (for both mobile and desktop) \
is that it will generate response of the length specified by context size. So, if you set it to 4096, it will generate 4096 tokens. \
As a result, it will either cut its answers, or generate random garbage to fill the space. \

# 4 BERT and custom models with Llama.cpp
This is where things are starting to get worse and worse. \
First of all, BERTs are TECHNICALLY supported by Llama.cpp, but I will describe how to run one in a second. \
Do you remember the .gguf format? It is not a commot format, it was created by the author of Llama.cpp (to replace .ggml format but that's another story).  \
Every model you want to run with Llama.cpp has to be converted to this format. TheBloke guy has a few of them, but if we want to \
fine-tune our onw model and run it , we need to somehow convert it to this format. \
Now, there is a function that you can use for this: 'convert_hf_to_gguf.py', but it wasn't working for me when used with google/bert-base-uncased. \
I was getting an error that this type of bert (specifically 'BertForMaskedLM') is not supported. \
There is another function called 'convert_hf_to_gguf_update.py'. After some digging it turned out that it has a hardcoded list of models that \
llama.cpp supports. If you want to use a new one, you have to add it to this list, run this function, and as a result you will get \
a python function called 'get_vocab_base_pre()' that you should paste in the place of the old one in the 'convert_hf_to_gguf.py'. \
I did that, and except the fact that I had to remove all other models because half of them are restricted now so they cannot be downloaded and script crashes, \
it seemed that it worked. But, when I tried to convert the model, I was still getting an error that the model is not supported. \
After even more digging I found out why: For every type of model, there is a DEDICATED implementation of the conversion function. \
There is one for BertModel, but BertModel != BertForMaskedLM (I tried to force the code to treat it as BertModel, but it didn't work). \
So, it seems that if we want to use it with our own models, we have to write our own conversion function. \

 

Still, I was curious how does the pipeline work, so I converted and run on of the two BERT models supported by Llama.cpp: \
https://huggingface.co/BAAI/bge-small-en-v1.5
https://github.com/FlagOpen/FlagEmbedding
The main problem with this model is that it only returns numerical embeddings, so we would probably manually have to convert them to words. \
Anyway, here are the steps to convert and run this model:
1. Download the snapshot of the model: to do this, you can use the python script below:
```python
from huggingface_hub import snapshot_download

snapshot_download("BAAI/bge-small-en-v1.5")
```
2. Get the snapshot's path: this snapshot will be saved in the hugging face subfolder of the ~/.cache folder. For me it was
```bash
/home/gustaw/.cache/huggingface/hub/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a'
```
3. Setup python env: in llama.cpp, run the following commands:
```bash
python -m venv .
source .venv/bin/activate
```
This will create and start venv for you. Now, enter 'requirements' dir, and run:
```bash
pip install -r requirements-all.txt
```
Now get back to the root of llama.cpp.
4. (Optional): Run 'convert_hf_to_gguf_update.py' to see how it behaves. To run it, call:
```bash
python convert_hf_to_gguf_update.py <your_hugging_face_token>
```
For the Bert-only version, I got the following output:
![update](./ResearchImages/convert_update.png)
5. Run the conversion: now, run the following command:
```bash
python convert_hf_to_gguf.py -m <path_to_snapshot>
```
For me, it was:
```bash
python convert_hf_to_gguf.py -m /home/gustaw/.cache/huggingface/hub/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a
```
Output:
![convert](./ResearchImages/convert.png)
As you can see, it said where it stored the model.
6. Quanitze the model: now, run the following command:
```bash
./build/bin/llama-quantize -m <path_to_converted_model.gguf> <quantization_type>
```
For me, it was:
```bash
./build/bin/llama-quantize /home/gustaw/.cache/huggingface/hub/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a-33M-5c38ec7c405ec4b44b94cc5a9bb96e735b38267a-F16.gguf Q8_0
```
Output: \
![quantize](./ResearchImages/quant.png) \
As you can probably deduce from my command, I used Q8_0 quantization. \
7. Run the model: now, run the following command:
```bash
sudo build/bin/llama-embedding --batch-size 4096 --ctx-size 2048 -m <path-to_quantized_model> -p <prompt>
```
Again, for me it was:
```bash
sudo build/bin/llama-embedding --batch-size 4096 --ctx-size 2048 -m /home/gustaw/.cache/huggingface/hub/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a/ggml-model-Q8_0.gguf -p "What is the capital of France?"
```
Path of the quantized model can be found in the output of the quantization script. \
Two things: first, without sudo this script won't have provoleges to create threads needed for calculations.\
Second, batch size has to be bigger than context size, otherwise you will get a cpp assertion error. \
Output: \
![bert](./ResearchImages/converted_run.png)

# 5. Conclusion
Llama.cpp is a great tool for local inference of LLMs (and other supported types of models). But only of the supported ones.
Idea of usage of this tool comes from this paper: https://arxiv.org/html/2410.03613v1. What quickly became obvious for me is that \
in ths research the model used for testing was llama-2.7B, which is basically a model from the tutorial. \
Adapting it for our own needs may not necessary require a lot of code (I'm talking about adding a code for conversion of the model), \
but it may be a lot of work to actually learn what needs to be done. \
To summarize, I think that it is technically possible to use Llama.cpp for our needs, but it will be challenging, \
and so I would leave it as a last resort resulution. \

## Appendix: Resources used/related to this research
https://github.com/ggerganov/llama.cpp \
https://www.reddit.com/r/LocalLLaMA/comments/14rncnb/local_llama_on_android_phone/ \
https://www.1a-insec.net/w/21-adding-semantic-search/ \
https://github.com/skeskinen/bert.cpp \
https://www.youtube.com/watch?v=jOEu0PE4ozM

In [ ]:
# Take Termux from f-droid

# pkg install cmake
# pkg install libllvm
# git clone llamacpp
# cd llamacpp
# cmake -B build
# cmake --build build --config Release

# cd ..
# curl -L -o llama-2-7b-chat.Q2_K.gguf https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q2_K.gguf
# cd llamacpp
# ./build/bin/llama-server -m ~/tinyllama-1.1b-chat-v0.3.Q8_0.gguf -c 4096


# Works for both mobile and desktop
# https://github.com/ggerganov/llama.cpp/tree/master/examples/server has a server with API,
# so technically we can make app, deploy server somewhere on mobile and app would call the local server

# has some android demo to checkout, do it

https://www.reddit.com/r/LocalLLaMA/comments/14rncnb/local_llama_on_android_phone/
